# BaseLine

### Import library

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import re
from datetime import datetime
import os

In [2]:
def process_mutation(mutation):
    if pd.isna(mutation):
        return 'Unknown', 0
    
    if isinstance(mutation, (int, float)):
        return 'Numeric', int(mutation) if mutation.is_integer() else mutation
    
    if not isinstance(mutation, str):
        return 'Unknown', 0
    
    if mutation == 'WT':
        return 'WT', 0
    
    match = re.match(r'([A-Z])(\d+)([A-Z*])', mutation)
    if match:
        original, position, mutated = match.groups()
        
        if mutated == '*':
            return 'NN', int(position)  # 무의미변이
        elif original == mutated:
            return 'SM', int(position)  # 동의어변이
        elif 'fs' in mutation:
            return 'FS', int(position)  # 프레임시프트변이
        else:
            return 'MS', int(position)  # 미스센스변이
    
    mutations = mutation.split(';')
    if len(mutations) == 2:
        type1, pos1 = process_mutation(mutations[0])
        type2, pos2 = process_mutation(mutations[1])
        return f"{type1};{type2}", max(pos1, pos2)
    
    return 'Unknown', 0

def preprocess_data(df, is_train=True):
    new_columns = ['ID']
    if is_train:
        new_columns.append('SUBCLASS')
    
    for column in df.columns:
        if column not in ['ID', 'SUBCLASS']:
            new_columns.extend([column, f"{column}_N"])
    
    new_df = pd.DataFrame(columns=new_columns)
    new_df['ID'] = df['ID']
    if is_train:
        new_df['SUBCLASS'] = df['SUBCLASS']
    
    for column in df.columns:
        if column not in ['ID', 'SUBCLASS']:
            mutation_type = []
            mutation_position = []
            for mutation in df[column]:
                m_type, m_pos = process_mutation(mutation)
                mutation_type.append(m_type)
                mutation_position.append(m_pos)
            new_df[column] = mutation_type
            new_df[f"{column}_N"] = mutation_position
    
    return new_df

def create_timestamp_folder(base_path='./submissions'):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_path = os.path.join(base_path, timestamp)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

### Load Data

In [3]:
train = pd.read_csv('./data/open/train.csv')
test = pd.read_csv("./data/open/test.csv")

### Data Preprocessing

In [4]:
# 데이터 전처리
train_processed = preprocess_data(train, is_train=True)
test_processed = preprocess_data(test, is_train=False)

# 특성 엔지니어링
def engineer_features(df):
    feature_df = pd.DataFrame()
    for col in df.columns:
        if col.endswith('_N'):
            continue
        if col not in ['ID', 'SUBCLASS']:
            feature_df[f'{col}_WT'] = (df[col] == 'WT').astype(int)
            feature_df[f'{col}_MS'] = (df[col] == 'MS').astype(int)
            feature_df[f'{col}_NN'] = (df[col] == 'NN').astype(int)
            feature_df[f'{col}_FS'] = (df[col] == 'FS').astype(int)
            feature_df[f'{col}_SM'] = (df[col] == 'SM').astype(int)
            feature_df[f'{col}_Unknown'] = (df[col] == 'Unknown').astype(int)
            feature_df[f'{col}_Numeric'] = (df[col] == 'Numeric').astype(int)
            feature_df[f'{col}_Position'] = df[f'{col}_N']
    return feature_df

X_train = engineer_features(train_processed)
X_test = engineer_features(test_processed)
y_train = train_processed['SUBCLASS']

In [10]:
# 타겟 변수 인코딩
le_subclass = LabelEncoder()
y_train = le_subclass.fit_transform(y_train)

# 변환된 레이블 확인
print("변환된 레이블:")
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

변환된 레이블:
원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25


### Model define & Train

In [11]:
# XGBoost 모델 정의 및 훈련
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

In [12]:
model.fit(X_train, y_train)

c:\Users\wjdwn\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:31:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

### Inference

In [13]:
predictions = model.predict(X_test)

In [14]:
original_labels = le_subclass.inverse_transform(predictions)

### Submisson

In [22]:
submission = pd.read_csv("./data/open/sample_submission.csv")
submission["SUBCLASS"] = original_labels

# 타임스탬프 폴더 생성
submission_folder = create_timestamp_folder()

submission_path = os.path.join(submission_folder, 'submission.csv')
submission.to_csv(submission_path, encoding='UTF-8-sig', index=False)